### Goal of this IPYNB file is to create a table relating an ambience categorical vector variable to its integer encoding. (e.g. a vector is 1120 then the first ambience attribute, classy, is false meaning the business is not "classy" the second attribute is false meaning casual is false since casual is the second attribute, the third attribute hipster is 2 meaning it has hipster ambience, and the last one is 0 meaning no information on intimate. Thus, 1120 is interpreted in 3-ary format as a number).

In [174]:
import pandas as pd
#def CustomParser(data):
#    import json
#    import numpy as np
#    if (data == np.nan):
#        return json.loads("{}")
#    j1 = json.loads(data)
#    return j1
df = pd.read_csv('/Users/amroa/Downloads/yelpdata/yelp-csv/yelp_academic_dataset_business.csv')
df.dropna(subset = ['business_id','stars', 'review_count', 'name', 'is_open'], inplace= True)

In [175]:
df['attributes']

0                                                        {}
1         {'GoodForMeal': "{'dessert': False, 'latenight...
2         {'BusinessParking': "{'garage': False, 'street...
3         {'BusinessParking': "{'garage': False, 'street...
4                                                        {}
                                ...                        
192604    {'Ambience': "{'romantic': False, 'intimate': ...
192605                                                   {}
192606                                                  NaN
192607    {'BusinessParking': "{'garage': False, 'street...
192608                                                  NaN
Name: attributes, Length: 192609, dtype: object

In [176]:
# Examining the graph above you can conclude 
print(df.info)
import numpy as np
df = pd.DataFrame(df[['business_id','attributes']], columns = ['business_id', 'attributes'])
df = df.dropna()
df

<bound method DataFrame.info of                                          address  \
0                    2818 E Camino Acequia Drive   
1                           30 Eglinton Avenue W   
2                   1210 8th Street SW, Unit 220   
3                    6955 N Durango Dr, Ste 1116   
4                              4 E University Dr   
...                                          ...   
192604                521 St. Catherine Street W   
192605                       244 Victoria Street   
192606                       10210 South 50th Pl   
192607  6670 S Tenaya Way, Ste 120, The Body Spa   
192608                 14050 N 83rd Ave, Ste 290   

                                               attributes  \
0                                                      {}   
1       {'GoodForMeal': "{'dessert': False, 'latenight...   
2       {'BusinessParking': "{'garage': False, 'street...   
3       {'BusinessParking': "{'garage': False, 'street...   
4                                     

,business_id,attributes
0,1SWheh84yJXfytovILXOAQ,{}
1,QXAEGFB4oINsVuTFxEYKFQ,"{'GoodForMeal': ""{'dessert': False, 'latenight..."
2,fcXOEZdXYeZqnQ3lGlOXmg,"{'BusinessParking': ""{'garage': False, 'street..."
3,f2ZWZPENViL92BrFsIgR6w,"{'BusinessParking': ""{'garage': False, 'street..."
4,6KgGE8B1RsR7jc9R5nuH0Q,{}
...,...,...
192601,zsrPJ7WD2OTrH1sbg2S7gg,{}
192603,Ul12enCzNT0G_jj5TCvJ8Q,{}
192604,wWV90q48ugyxyXW6gZav8w,"{'Ambience': ""{'romantic': False, 'intimate': ..."
192605,phSaT3I_y6To1J2OdeFSUQ,{}


In [142]:
#the following is a guide on how to parse the fuck out of a JSON variable

In [143]:
filtamb = df['attributes'].str.contains("Ambience")
df = df[filtamb]

In [144]:
toParse = df.loc[1].attributes.split('Ambience')

In [145]:
beginIndex = toParse[1].find('{')
beginIndex

4

In [146]:
endIndex = toParse[1].find('}')
endIndex

162

In [147]:
ambience_fields = toParse[1]
ambience_fields = ambience_fields[beginIndex:(endIndex+1)]
ambience_fields

"{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}"

In [148]:
splitted_fields = ambience_fields.split(': ')
splitted_fields

["{'romantic'",
 "False, 'intimate'",
 "False, 'classy'",
 "False, 'hipster'",
 "False, 'divey'",
 "False, 'touristy'",
 "False, 'trendy'",
 "False, 'upscale'",
 "False, 'casual'",
 'True}']

In [149]:
splitted_fields[0] = (splitted_fields[0])[1:]
splitted_fields[-1] = (splitted_fields[-1])[0:4]
splitted_fields

["'romantic'",
 "False, 'intimate'",
 "False, 'classy'",
 "False, 'hipster'",
 "False, 'divey'",
 "False, 'touristy'",
 "False, 'trendy'",
 "False, 'upscale'",
 "False, 'casual'",
 'True']

In [150]:
mycleanlist = list()
for i in range(0, len(splitted_fields) -1):
    commasplitted1 = splitted_fields[i].split(', ')
    commasplitted2 = splitted_fields[i+1].split(', ')
    name_fields = commasplitted1[-1] #e.g. touristy
    name_value_none = 2*int(commasplitted2[0] == 'None')
    name_value_true = 1*int(commasplitted2[0] == 'True')
    name_value_false = 0
    mycleanlist.append((name_fields[1: -1], name_value_none + name_value_true + name_value_false))
mycleanlist

[('romantic', 0),
 ('intimate', 0),
 ('classy', 0),
 ('hipster', 0),
 ('divey', 0),
 ('touristy', 0),
 ('trendy', 0),
 ('upscale', 0),
 ('casual', 1)]

In [151]:
#given a list of tuples get the AID (ambience ID) (in base 3 so 6561 = 3^8, 2187 = 3^7, etc... and 0 encodes None, 2 encodes True, 1 encodes False)
weights = {'romantic': 6561, 'intimate': 2187, 'classy': 729, 'hipster': 243, 'divey': 81,
          'touristy': 27, 'trendy': 9, 'upscale': 3, 'casual': 1}
aid = 0
for i in mycleanlist:
    aid = aid + weights[i[0]]*i[1]
aid

1

In [152]:
def attributeAmbienceToAid(attributes):
    splitAtAmbience = str(attributes).split("'Ambience': ")
    beginIndex      = splitAtAmbience[1].find('{')
    endIndex        = splitAtAmbience[1].find('}')
    ambience_fields = (splitAtAmbience[1])[beginIndex:(endIndex+1)]
    splitted_fields = ambience_fields.split(': ')
    
    #trim out the annoying braces
    splitted_fields[0] = (splitted_fields[0])[1:]
    splitted_fields[-1] = (splitted_fields[-1])[0:4]
    if((splitAtAmbience[1]).startswith("'None'")):
        return -1
    
    # get the aid
    aid = 0
    for i in range(0, len(splitted_fields) -1):
        commasplitted1 = splitted_fields[i].split(', ')
        commasplitted2 = splitted_fields[i+1].split(', ')
        name_fields = commasplitted1[-1] #e.g. touristy
        name_value = 2*int(commasplitted2[0] == 'True') + 1*int(commasplitted2[0] == 'False')
        aid = aid + weights[name_fields[1: -1]]*name_value
    return aid
    

In [153]:
df.loc[1].attributes

'{\'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'NoiseLevel\': "u\'loud\'", \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}"}'

In [154]:
#test this method 
x = attributeAmbienceToAid(df.loc[1].attributes)
print('done with value '+str(x))

done with value 9842


In [155]:
for i in range(0,20):
    print('\n \n')
    print(df.iloc[i].attributes)


 

{'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'NoiseLevel': "u'loud'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}"}

 

{'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}", 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'NoiseLevel': "u'average'"}

 

{'NoiseLevel': "'average'", 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lu

In [156]:
df['attributes'] = df['attributes'].apply(attributeAmbienceToAid)
print('done')

done


/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [157]:
df.head(20)

,business_id,attributes
1,QXAEGFB4oINsVuTFxEYKFQ,9842
11,YK8aOugv9mLrzBgJKP9biA,9759
18,8Wm2NMAoukBC2wQR49E4ng,9842
21,8HAsNLoSr3xi5rwQretp3w,9840
24,b01JHXXVU9XuUuFMGWDMJw,9840
25,JGM-SxW_vayu0fBch-U2FQ,9840
31,GWCd4xlTdOwcaaSSu9b9XQ,9842
36,omfSvRsQVhJkU_yB8udhIw,9759
37,N0Tk9ORpX-gtbPNR0WoznQ,9840
49,rsiW1dz9FD1EfNPoDfaDDQ,9842


In [158]:
len(df)

48079

In [159]:
df.rename(columns={'attributes':'AID'}, inplace=True)

/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [160]:
df

,business_id,AID
1,QXAEGFB4oINsVuTFxEYKFQ,9842
11,YK8aOugv9mLrzBgJKP9biA,9759
18,8Wm2NMAoukBC2wQR49E4ng,9842
21,8HAsNLoSr3xi5rwQretp3w,9840
24,b01JHXXVU9XuUuFMGWDMJw,9840
...,...,...
192590,2_aYWG3cVnAXH6yHzGu05Q,9840
192592,Gfa85o6JuT9uBqi9fqC-UA,9842
192594,8QWPlVQ6D-OExqXoaD2Z1g,9840
192595,0OvM4_xU5cJYCo0bsbVEQw,9842


In [161]:
df.replace(-1, np.nan, inplace=True)

/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [162]:
df = df.dropna()

In [163]:
df

,business_id,AID
1,QXAEGFB4oINsVuTFxEYKFQ,9842.0
11,YK8aOugv9mLrzBgJKP9biA,9759.0
18,8Wm2NMAoukBC2wQR49E4ng,9842.0
21,8HAsNLoSr3xi5rwQretp3w,9840.0
24,b01JHXXVU9XuUuFMGWDMJw,9840.0
...,...,...
192590,2_aYWG3cVnAXH6yHzGu05Q,9840.0
192592,Gfa85o6JuT9uBqi9fqC-UA,9842.0
192594,8QWPlVQ6D-OExqXoaD2Z1g,9840.0
192595,0OvM4_xU5cJYCo0bsbVEQw,9842.0


In [164]:
df['AID'] = df['AID'].astype(int)
#save df to csv here
df

/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,business_id,AID
1,QXAEGFB4oINsVuTFxEYKFQ,9842
11,YK8aOugv9mLrzBgJKP9biA,9759
18,8Wm2NMAoukBC2wQR49E4ng,9842
21,8HAsNLoSr3xi5rwQretp3w,9840
24,b01JHXXVU9XuUuFMGWDMJw,9840
...,...,...
192590,2_aYWG3cVnAXH6yHzGu05Q,9840
192592,Gfa85o6JuT9uBqi9fqC-UA,9842
192594,8QWPlVQ6D-OExqXoaD2Z1g,9840
192595,0OvM4_xU5cJYCo0bsbVEQw,9842


In [165]:
df.drop_duplicates(subset =['business_id'], 
                     keep = 'first', inplace = True) 
df

/Users/amroa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,business_id,AID
1,QXAEGFB4oINsVuTFxEYKFQ,9842
11,YK8aOugv9mLrzBgJKP9biA,9759
18,8Wm2NMAoukBC2wQR49E4ng,9842
21,8HAsNLoSr3xi5rwQretp3w,9840
24,b01JHXXVU9XuUuFMGWDMJw,9840
...,...,...
192590,2_aYWG3cVnAXH6yHzGu05Q,9840
192592,Gfa85o6JuT9uBqi9fqC-UA,9842
192594,8QWPlVQ6D-OExqXoaD2Z1g,9840
192595,0OvM4_xU5cJYCo0bsbVEQw,9842


In [166]:
df_amb = pd.DataFrame(df['AID'])
df_amb

,AID
1,9842
11,9759
18,9842
21,9840
24,9840
...,...
192590,9840
192592,9842
192594,9840
192595,9842


In [167]:
#map the ternary strings to list
def bitcountToRomBoolean(aid, ambfield):
    leftShift = aid//weights[ambfield]
    value__ = (leftShift)%3
    return value__
    
df_amb['romantic'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'romantic'))
df_amb['casual'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'casual'))
df_amb['trendy'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'trendy'))

df_amb['touristy'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'touristy'))
df_amb['divey'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'divey'))
df_amb['hipster'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'hipster'))

df_amb['intimate'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'intimate'))
df_amb['classy'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'classy'))
df_amb['upscale'] = df_amb['AID'].apply(lambda x : bitcountToRomBoolean(x, 'upscale'))

In [168]:
df_amb

,AID,romantic,casual,trendy,touristy,divey,hipster,intimate,classy,upscale
1,9842,1,2,1,1,1,1,1,1,1
11,9759,1,0,1,1,0,1,1,1,1
18,9842,1,2,1,1,1,1,1,1,1
21,9840,1,0,1,1,1,1,1,1,1
24,9840,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
192590,9840,1,0,1,1,1,1,1,1,1
192592,9842,1,2,1,1,1,1,1,1,1
192594,9840,1,0,1,1,1,1,1,1,1
192595,9842,1,2,1,1,1,1,1,1,1


In [169]:
#remove duplicates
df_amb.drop_duplicates(subset ="AID", 
                     keep = 'first', inplace = True) 
df_amb

,AID,romantic,casual,trendy,touristy,divey,hipster,intimate,classy,upscale
1,9842,1,2,1,1,1,1,1,1,1
11,9759,1,0,1,1,0,1,1,1,1
21,9840,1,0,1,1,1,1,1,1,1
88,10094,1,2,2,1,1,2,1,1,1
103,19328,2,2,2,1,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...
178247,13013,1,2,2,1,1,2,2,2,2
180186,19344,2,0,1,2,1,1,2,2,1
183080,16652,2,2,2,1,1,2,1,1,0
187914,16334,2,2,2,1,0,1,1,1,2


In [170]:
#write to csv the df_amb here

In [171]:
df_amb.to_csv('ambienceAB.csv', index = False)

In [172]:
df.to_csv('hasAmbienceAB.csv', index= False)